In [2]:
import pandas as pd 
import numpy as np
import random
from citipy import citipy
from datetime import datetime 
import requests
from config_api import weather_apiKey as wkey

In [3]:

# Create list for weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Create basic url for API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + wkey

Beginning Data Retrieval     
-----------------------------


In [4]:
# Create 2000 random latitude and longitude coordinates 
lats = np.random.uniform(-90, 90, size = 2000)
lngs = np.random.uniform(-180, 180, size = 2000)
lats_lngs = zip(lats, lngs)
lats_lngs

# Add coordinates to a list 
latlngs_lst = list(lats_lngs)

In [5]:
# Create list for cities 
cities = []

# Use citipy to find nearest cities using coordinate list
for latlng in latlngs_lst:
    city = citipy.nearest_city(latlng[0],latlng[1])
    city_name = city.city_name
    country_name = city.country_code
    
    # Find unique cities to add to cities list
    if city_name not in cities:
        cities.append(city_name)
    
    # See cities and contries
    # print(f'{city_name}, {country_name}')

# Count amount of unique cities
len(cities)

735

In [6]:

# Loop through city Lists 
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 
    if (i % 50 == 0 and i >= 50):
        set_count += 1 
        record_count = 1 
    
    # Create URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log URL, set, and record numbers for each city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count += 1

# Run an API request for each of the cities
    try:
       
        # Parse the JSON data
        city_weather = requests.get(city_url).json()

        # Compile needed data
        city_country = city_weather['sys']['country']
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_hum = city_weather['main']['humidity']
        city_cloud = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_desc = city_weather['weather'][0]['description']


        # Fill City Data list 
        city_data.append({"City": city.title(),
                         "Country": city_country,
                         "Date": city_date,
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_hum,
                         "Cloudiness": city_cloud,
                         "Wind Speed": city_wind,
                         "Current Weather": city_desc
                         })
        
    #If an error occurs
    except: 
        print("--- City not found")
        pass

    # Complete Data Retrieval    
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | saint george
Processing Record 2 of Set 1 | general cepeda
Processing Record 3 of Set 1 | mataura
Processing Record 4 of Set 1 | shahrud
Processing Record 5 of Set 1 | mitu
Processing Record 6 of Set 1 | big spring
Processing Record 7 of Set 1 | castro
Processing Record 8 of Set 1 | ilulissat
Processing Record 9 of Set 1 | tuatapere
Processing Record 10 of Set 1 | bredasdorp
Processing Record 11 of Set 1 | montrose
Processing Record 12 of Set 1 | georgetown
Processing Record 13 of Set 1 | vanimo
Processing Record 14 of Set 1 | bremervorde
Processing Record 15 of Set 1 | rikitea
Processing Record 16 of Set 1 | linkoping
Processing Record 17 of Set 1 | ocean city
Processing Record 18 of Set 1 | tarudant
--- City not found
Processing Record 19 of Set 1 | busselton
Processing Record 20 of Set 1 | sibolga
Processing Record 21 of Set 1 | yomitan
--- City not found
Processing Record 22 of Set 1 | taolanaro
--- City not found
Processing Record 23 of Set 1 | east 

In [7]:

# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,Saint George,US,2021-07-25 20:15:14,37.1041,-113.5841,96.80,34,1,1.01,clear sky
1,General Cepeda,MX,2021-07-25 20:15:15,25.3833,-101.4500,85.41,21,2,15.12,clear sky
2,Mataura,NZ,2021-07-25 20:15:15,-46.1927,168.8643,46.47,92,100,3.38,overcast clouds
3,Shahrud,IR,2021-07-25 20:15:15,36.4182,54.9763,77.61,52,0,10.18,clear sky
4,Mitu,CO,2021-07-25 20:15:15,1.1983,-70.1733,77.32,80,81,3.38,broken clouds
5,Big Spring,US,2021-07-25 20:15:15,32.2504,-101.4787,99.28,36,1,3.00,clear sky
6,Castro,BR,2021-07-25 20:13:11,-24.7911,-50.0119,65.03,61,2,3.85,clear sky
7,Ilulissat,GL,2021-07-25 20:15:16,69.2167,-51.1000,53.62,37,100,9.22,overcast clouds
8,Tuatapere,NZ,2021-07-25 20:15:16,-46.1333,167.6833,46.81,95,100,4.74,overcast clouds
9,Bredasdorp,ZA,2021-07-25 20:15:16,-34.5322,20.0403,52.32,51,100,4.68,overcast clouds


In [8]:
# Create the output file 
output_data_file = '../weather_data/cities.csv'

# Export the City_Data into a CSV
city_data_df.to_csv('WeatherPy_Database.csv', index_label='City_ID')